# Metric Evaluation for 3D Volumes
This notebook computes Chamfer Distance and Volumetric IoU for a single predicted model against ground truth volumes.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import mrcfile
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

In [ ]:
def load_mrc(file_path):
    with mrcfile.open(file_path, permissive=True) as mrc:
        return np.array(mrc.data, dtype=np.float16)

def apply_mask(volume, mask):
    return volume * mask if mask is not None else volume

def get_voxel_coords(volume, threshold, grid_size=128, voxel_size=4.5):
    coords = np.argwhere(volume >= threshold)
    return (coords - np.array([grid_size // 2]*3)) * voxel_size

def chamfer_distance(gt_vol, pred_vol, pred_thresh, mask=None, grid_size=128, voxel_size=4.5):
    gt_vol = apply_mask(gt_vol, mask)
    pred_vol = apply_mask(pred_vol, mask)
    gt_coords = get_voxel_coords(gt_vol, 0.05, grid_size, voxel_size)
    pred_coords = get_voxel_coords(pred_vol, pred_thresh, grid_size, voxel_size)
    if gt_coords.size == 0 or pred_coords.size == 0:
        return np.nan
    gt_tree, pred_tree = cKDTree(gt_coords), cKDTree(pred_coords)
    d1, _ = pred_tree.query(gt_coords)
    d2, _ = gt_tree.query(pred_coords)
    return np.mean(d1) + np.mean(d2)

def volumetric_iou(gt_vol, pred_vol, pred_thresh, mask=None):
    gt_vol = apply_mask(gt_vol, mask)
    pred_vol = apply_mask(pred_vol, mask)
    gt_binary = (gt_vol >= 0.05)
    pred_binary = (pred_vol >= pred_thresh)
    inter = np.logical_and(gt_binary, pred_binary).sum()
    union = np.logical_or(gt_binary, pred_binary).sum()
    return inter / union if union > 0 else 0

In [ ]:
# CONFIGURATION
gt_dir = "/scratch/gpfs/ZHONGE/mj7341/data/cryobench/IgG-RL/vols/128_org"
pred_dir = "/scratch/gpfs/ZHONGE/jg1427/results/IgG-RL/noisy_crop/vols"
thresholds = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
mask_path = "/scratch/gpfs/ZHONGE/mj7341/data/cryobench/IgG-RL/init_mask/mask.mrc"
grid_size = 128
voxel_size = 4.5

# EVALUATION
results = []
pred_files = sorted(glob.glob(os.path.join(pred_dir, "vol_*.mrc")))

for pred_file in pred_files:
    basename = os.path.basename(pred_file)
    idx = int(basename.split('_')[-1].split('.')[0])
    gt_file = os.path.join(gt_dir, f"vol_{idx:03d}.mrc")
    if not os.path.exists(gt_file):
        print(f"Missing GT for {basename}, skipping...")
        continue
    gt_vol = load_mrc(gt_file)
    pred_vol = load_mrc(pred_file)
    mask = load_mrc(mask_path) if mask_path else None

    for t in thresholds:
        cd = chamfer_distance(gt_vol, pred_vol, t, mask, grid_size, voxel_size)
        iou = volumetric_iou(gt_vol, pred_vol, t, mask)
        results.append({
            "Volume": basename,
            "Threshold": t,
            "Chamfer Distance": cd,
            "IoU": iou
        })

df = pd.DataFrame(results)
df.head()

In [ ]:
# PLOTTING
grouped = df.groupby("Threshold").agg({
    "Chamfer Distance": ["mean", "std"],
    "IoU": ["mean", "std"]
}).reset_index()

fig, axs = plt.subplots(1, 2, figsize=(12, 5))
thresholds = grouped["Threshold"]

cd_mean = grouped["Chamfer Distance"]["mean"]
cd_std = grouped["Chamfer Distance"]["std"]
axs[0].plot(thresholds, cd_mean, color='purple', label="Chamfer Distance")
axs[0].fill_between(thresholds, cd_mean - cd_std, cd_mean + cd_std, alpha=0.3)
axs[0].set_title("Chamfer Distance vs Threshold")

iou_mean = grouped["IoU"]["mean"]
iou_std = grouped["IoU"]["std"]
axs[1].plot(thresholds, iou_mean, color='teal', label="IoU")
axs[1].fill_between(thresholds, iou_mean - iou_std, iou_mean + iou_std, alpha=0.3)
axs[1].set_title("IoU vs Threshold")

for ax in axs:
    ax.set_xlabel("Isosurface Threshold")
    ax.set_ylabel("Metric")
    ax.grid(True)

plt.tight_layout()
plt.show()